In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, concatenate
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D
from keras.layers import Add, BatchNormalization, Activation, CuDNNLSTM, Dropout
from keras.layers import *
from keras.models import *
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import gc
from sklearn import metrics
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix

In [ ]:
train_df = pd.read_csv('../input/train.csv')
#train_df["question_text"] = train_df["question_text"].map(lambda x: clean_text(x))

test_df = pd.read_csv('../input/test.csv')
#test_df["question_text"] = test_df["question_text"].map(lambda x: clean_text(x))

In [ ]:
X_train = train_df["question_text"].fillna("na").values
X_test = test_df["question_text"].fillna("na").values
y = train_df["target"]

In [ ]:
maxlen = 70
max_features = 50000
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print("finished!")

In [ ]:
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    TIME_STEPS = inputs.shape[1].value
    SINGLE_ATTENTION_VECTOR = False
    
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values()) # 二维数组 2096016*300
print(all_embs.shape[0])
emb_mean,emb_std = all_embs.mean(), all_embs.std()  # scalar 均值，标准差
print("mean=",emb_mean)
embed_size = all_embs.shape[1]  # 300

word_index = tokenizer.word_index  # 单词对应的整数编号形成的列表
nb_words = min(max_features, len(word_index))  # 只取两者中较小者的单词数量
# 结合下面的代码。对于embeddings中没有的单词，使用随机初始化的词向量
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() # 内存空间清理
print("finished!")

In [ ]:
EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())  # 999994*300
print(all_embs.shape[0])
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_2[i] = embedding_vector
del embeddings_index; gc.collect()
print("finished!")

In [ ]:
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values()) # 1703755*300
print(all_embs.shape[0])
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector
        
del embeddings_index; gc.collect()   
print("finished!")

In [ ]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3), axis=1)  
del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3
gc.collect()
print(np.shape(embedding_matrix))
print("finished!")

In [ ]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y, test_size = 0.1, random_state=42)

In [ ]:
def model_conv1d():
    filters = 128
    
    inp = Input(shape=(maxlen, ))
    embed = Embedding(max_features, embed_size * 3, weights=[embedding_matrix], trainable=False)(inp)
    x = embed
    
    x = Conv1D(filters, 1, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Conv1D(filters, 2, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Conv1D(filters, 3, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    x = Conv1D(filters, 5, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    #x = Flatten()(x)
    x = GlobalAveragePooling1D()(x)
    
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])    
    return model

In [ ]:
MODEL = model_conv1d()
MODEL.summary()

batch_size = 2048
epochs = 1

early_stopping = EarlyStopping(patience=3, verbose=1, monitor='val_loss', mode='min')
model_checkpoint = ModelCheckpoint('./model_conv1d.model', save_best_only=True, verbose=1, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=0.0001, verbose=1)

hist = MODEL.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
MODEL.save('./model_conv1d.h5')

In [ ]:
pred_val_y_3 = MODEL.predict([X_val], batch_size=1024, verbose=1)
thresholds = []
f1_list = list()
thre_list = np.arange(0.1, 0.501, 0.01)
for thresh in thre_list:
    thresh = np.round(thresh, 2)
    acc = metrics.accuracy_score(y_val, (pred_val_y_3 > thresh).astype(int))
    f1 = metrics.f1_score(y_val, (pred_val_y_3 > thresh).astype(int))
    f1_list.append(f1)
    pre = metrics.precision_score(y_val, (pred_val_y_3 > thresh).astype(int))
    recall = metrics.recall_score(y_val, (pred_val_y_3 > thresh).astype(int))
    thresholds.append([thresh, f1])
    print("acc score at threshold {0} is {1}".format(thresh, acc))
    print("F1 score at threshold {0} is {1}".format(thresh, f1))
    print("precision at threshold {0} is {1}".format(thresh, pre))
    print("recall at threshold {0} is {1}".format(thresh, recall))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh_3 = thresholds[0][0]
print("Best threshold: ", best_thresh_3)

plot_confusion_matrix(y_val, np.array(pd.Series(pred_val_y_3.reshape(-1,)).map(lambda x:1 if x>thre_list[np.argmax(f1_list)] else 0)))

y_pred_3 = MODEL.predict(x_test, batch_size=1024, verbose=True)

In [ ]:
# pred_val_y = (3*pred_val_y_2 + 4*pred_val_y_4 + 3*pred_val_y_5)/10
# pred_val_y = pred_val_y_3

# thresholds = []
# for thresh in np.arange(0.1, 0.501, 0.01):
#     thresh = np.round(thresh, 2)
#     res = metrics.f1_score(y_val, (pred_val_y > thresh).astype(int))
#     thresholds.append([thresh, res])
#     print("F1 score at threshold {0} is {1}".format(thresh, res))
    
# thresholds.sort(key=lambda x: x[1], reverse=True)
# best_thresh = thresholds[0][0]
# print("Best threshold: ", best_thresh)

In [ ]:
# y_pred = (3*y_pred_2 + 4*y_pred_4 + 3*y_pred_5)/10
y_pred = y_pred_3
y_te = (y_pred[:,0] > best_thresh_3).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)